In [100]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
import torch
import torch.nn as nn


[autoreload of feed_forward failed: Traceback (most recent call last):
  File "/Users/bayesian_monster/anaconda3/envs/DLA/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/bayesian_monster/anaconda3/envs/DLA/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/bayesian_monster/anaconda3/envs/DLA/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/bayesian_monster/DLA/asr_project_template/hw_asr/model/ConformerV1/feed_forward.py", line 4, in <module>
    from .activations import Swish
ImportError: attempted relative import wit

In [102]:
cenc = encoder.ConformerEncoder(
    input_dim=128,
    d_encoder=512,
    num_layers=3,
    num_heads_attention=8,
    ffl_exp_factor=2,
    conv_exp_factor=2,
    conv_kernel_size=32,
    ffl_dropout=0.1,
    conv_dropout=0.1,
    attention_dropout=0.1
)

In [103]:
batch = torch.randn(3, 128, 128)
batch.shape

torch.Size([3, 128, 128])

In [104]:
subsample = conv.ConvSubsample(in_chanels=128, out_chanels=512)
linear_proj = nn.Sequential(
            nn.Linear(512, 512),
            nn.Dropout(0.1)
        )

In [105]:
subsampled = subsample(batch, batch.shape)

In [106]:
lin = linear_proj(subsampled)

In [107]:
from feed_forward import ConformerFeedForwardLayer

In [108]:
ffl = ConformerFeedForwardLayer(512, 2, 0.1)

In [109]:
after_ffl = ffl(lin) + lin

In [110]:
attn = ConformerAttentionBlock(512, 8, 0.1)

In [111]:
after_attn = attn(after_ffl) + after_ffl

In [112]:
from conv import ConformerConvBlock

In [116]:
conv_ = ConformerConvBlock(in_chanels=512, exp_factor=2, kernel_size=31, padding=15, stride=1, dropout=0.1)

In [117]:
after_attn.shape

torch.Size([3, 31, 512])

In [118]:
after_conv = conv_(after_attn)


torch.Size([3, 1024, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])


In [93]:
ffl_2 = ConformerFeedForwardLayer(512, 2, 0.1)

In [95]:
shit = (ffl_2(after_conv) + after_conv)

In [96]:
shit

torch.Size([3, 31, 512])

In [176]:
class ConvSubsample(nn.Module):
    def __init__(self, in_chanels, out_chanels):
        super().__init__()
        self.subsampler = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_chanels, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU()
        )
    
    def forward(self, x):
        out = self.subsampler(x.unsqueeze(1))
        batch_size, channels, subsample_len, subsample_dim = out.shape

        out = out.permute(0, 2, 1, 3)
        out = out.contiguous().view(batch_size, subsample_len, channels * subsample_dim)
        return out
    


In [181]:
testing_jopa = ConvSubsample(1, 144)
linear = nn.Linear(144 * ((140 - 1)//2 - 1)//2, 144)

In [178]:
batch = torch.randn(3, 128, 140)
batch.shape

torch.Size([3, 128, 140])

In [186]:
out = testing_jopa(batch)
linear(out).shape

3 144 31 34
4896


torch.Size([3, 31, 144])

In [184]:
144 * (((140 - 1) // 2 - 1) // 2)

4896